In [ ]:
import json
import sqlite3
import os
import time
import paho.mqtt.client as mqtt

# Load config
with open("battery_machine_config.json") as f:
    machine_configs = json.load(f)

db_dir = "./machine_dbs"
os.makedirs(db_dir, exist_ok=True)

db_connections = {}

def setup_database(machine):
    machine_name = machine["machine_name"]
    db_path = os.path.join(db_dir, f"{machine_name}.db")
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    for step in machine["steps"]:
        table_name = step["step_name"]
        cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS [{table_name}] (
                timestamp TEXT,
                value REAL,
                mean REAL,
                variance REAL,
                process_time_min INTEGER,
                fatal_failure BOOLEAN
            )
        """)
    conn.commit()
    db_connections[machine_name] = conn

for machine in machine_configs:
    setup_database(machine)

# MQTT callback
def on_message(client, userdata, msg):
    try:
        payload = json.loads(msg.payload.decode())
        machine_id = payload["machine_id"]
        step_name = payload["step_name"]
        conn = db_connections.get(machine_id)
        if conn:
            cursor = conn.cursor()
            cursor.execute(f"""
                INSERT INTO [{step_name}] (timestamp, value, mean, variance, process_time_min, fatal_failure)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                payload["timestamp"],
                payload["value"],
                payload["mean"],
                payload["variance"],
                payload["process_time_min"],
                int(payload["fatal_failure"])
            ))
            conn.commit()
    except Exception as e:
        print(f"Error logging message: {e}")

# MQTT setup
client = mqtt.Client()
client.on_message = on_message
client.connect("localhost", 1883, 60)

for machine in machine_configs:
    for step in machine["steps"]:
        topic = f"factory/{machine['machine_name']}/{step['step_name']}"
        client.subscribe(topic)

print("Subscribed to all machine/step topics. Logging...")
client.loop_forever()
